In [ ]:
import polars as pl
from datetime import datetime
import get_and_store as gas
import polars_indicators as pi
from polars_indicators import IndicatorResult
DATE_COLUNN = "Date"

In [ ]:
symbols = ['A','AA']

In [ ]:
df = gas.get_symbols_df(symbols)
df = df.filter(pl.col('Date') >= datetime(2021, 1, 1))
df = df.collect()

In [ ]:
df

In [ ]:
sma25 = pi.simple_moving_average(df, days=25)
sma50 = pi.simple_moving_average(sma25.df, days=50)
df = sma50.df
df

In [ ]:
cross_up = pi.crossover_up(df, sma25.column, sma50.column)
cross_up.df

In [ ]:
#convert to open values
cross_up.df = cross_up.df.with_columns(pl.when(pl.col(cross_up.column)).then(pl.col("Open")).alias(cross_up.column))

In [ ]:
trail = pi.trailing_stop(cross_up.df, bars=2)
trail.df.filter(pl.col("Date").is_between(datetime(2022, 6, 10), datetime(2022, 6, 20)))

In [ ]:
ret = pi.create_trade_ids(trail.df, cross_up.column, trail.column)

In [ ]:
pi.summarize_trades(ret.df, ret.column, cross_up.column, trail.column)